In [1]:
pip install transformers datasets accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!python3 -m pip install --upgrade 'optree>=0.13.0

## Load a pre-trained model and tokenizer

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)  # 3 classes: positive, negative, neutral
tokenizer = AutoTokenizer.from_pretrained(model_name)

D:\Anaconda\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Load data

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("sentiment_product_reviews.csv")
df.head(2)

,comment,label
0,"Moderate performance, works as intended.",1
1,"The product is just okay, nothing special.",1


## Convert the DataFrame to a Hugging Face Dataset

In [3]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)
print(dataset)

Dataset({
    features: ['comment', 'label'],
    num_rows: 20000
})


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

D:\Anaconda\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [5]:
# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples["comment"], padding="max_length", truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
#print(tokenized_dataset[0])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [8]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

#  Configure TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="my_sentiment_analysis_experiment",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Disable W&B logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
